In [1]:
import numpy as np
import healpy as hp
import pandas as pd
import matplotlib.pyplot as plt
import tools
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
bin_ellmin, bin_ellmax, bin_width = 20, 7020, 20
bin_ells = np.arange(bin_ellmin, bin_ellmax+1, 1)
smooth_ellmin, smooth_ellmax, smooth_width = 30, 7000, 1
smooth_ells = np.arange(smooth_ellmin, smooth_ellmax+1, smooth_width)

In [3]:
lensed_cmb_alms_file = './maps/websky/original/lensed_alm.fits'
kap_file = './maps/websky/original/kap.fits'
ksz_lt_file = './maps/websky/original/ksz.fits'
ksz_ri_file = './maps/websky/original/ksz_patchy.fits'

In [4]:
# write lensed_cmb_alms_7000.fits
lensed_cmb_alms = hp.read_alm(lensed_cmb_alms_file)
hp.write_alm('maps/websky/alms/lensed_cmb_alms_%s.fits' %(smooth_ellmax), lensed_cmb_alms, \
             lmax=smooth_ellmax, overwrite=True)

In [5]:
# write kap_alms_7000.fits
kap_map =  hp.read_map(kap_file)
kap_alms = hp.map2alm(kap_map, lmax=smooth_ellmax)
hp.write_alm('maps/websky/alms/kap_alms_7000.fits', kap_alms, overwrite=True)

/global/homes/h/hongbo/.local/cori/3.7-anaconda-2019.07/lib/python3.7/site-packages/healpy/fitsfunc.py:352: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  "If you are not specifying the input dtype and using the default "


NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


In [6]:
ksz_type = 'lt'
if ksz_type == 'lt':
    ksz_file = ksz_lt_file
else:
    ksz_file = ksz_ri_file
ksz = hp.read_map(ksz_file)
ksz_alms = hp.map2alm(ksz, lmax=bin_ellmax)
hp.write_alm('./maps/websky/alms/ksz_%s_alms_%s.fits' %(ksz_type, smooth_ellmax), ksz_alms)

NSIDE = 4096
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


OSError: File './maps/websky/alms/ksz_lt_alms_7000.fits' already exists.

In [71]:
ksz_cls = hp.alm2cl(ksz_alms)
Smooth_ksz = tools.bin_smooth(bin_ells, ksz_cls[bin_ells], bin_width)
smooth_ksz_cls = Smooth_ksz.get_cls(smooth_ells)

In [75]:
# concatenate the ps and ells, ells strarts from 0
smooth_ells_con = np.concatenate([np.arange(0, smooth_ellmin, 1), smooth_ells])
smooth_ksz_cls_con = np.concatenate([np.zeros(smooth_ellmin), smooth_ksz_cls])

In [76]:
# write smooth_ksz_cls_con
df = pd.DataFrame({'ells':smooth_ells_con, 'cls':smooth_ksz_cls_con})
df.to_csv('./maps/websky/cls/smooth_ksz_%s_cls_%s.csv' %(ksz_type, smooth_ellmax), index=False)